## Steps for topic  modeling
- data loading
- data preprocesing
    - Tokenization
    - Words that have fewer than 3 characters are removed.
    - All stopwords are removed.
    - lemmatize
- Modeling


In [1]:
import pandas as pd
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [2]:
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

ModuleNotFoundError: No module named 'src'

In [ ]:
# now lets define the path for week 8 and week 9
week_8_path = "C:/Users/ok/Desktop/Week_0/week-0/data/channels/weeks/all-week8"
week_9_path = "C:/Users/ok/Desktop/Week_0/week-0/data/channels/weeks/all-week9"

#now lets extract the data from the all-week8 and all-week9 folders
week_8 = utils.slack_parser(week_8_path)
week_9 = utils.slack_parser(week_9_path)

data = pd.concat([week_8, week_9])


In [ ]:
# loading the data
data_csv = data.to_csv('data_csv.csv')